In [2]:
import sys, os
from os.path import join, dirname, abspath
import matplotlib.pyplot as plt
import numpy as np
plt.rcParams.update({
    "text.usetex": False,
    "ytick.minor.visible":True,
    "xtick.minor.visible":True,
    'xtick.direction': "in",
    'ytick.direction': "in"
})
outdir = "project_1_out"
os.makedirs(outdir,exist_ok=True)
def out(fname): return join(outdir,fname)
def savefig(plot_name): 
    plt.savefig(out(plot_name),bbox_inches="tight",dpi=250)

In [3]:
brown = "tab:brown"
green = "tab:green"

# Part 1

In [4]:
d0 = 100
p_carrying = 3000
p0 = p_carrying
# p0 = 1000
plant_r = 0.8

In [5]:
def linear_eating(p,dt):
    return (1.2/p_carrying)*p*dt

def linear_birth(p,dt):
    return (1.5/p_carrying)*p*dt

def exponential_eating(p,dt):
    k = 0.001 - np.log(dt)/p_carrying
    return 1.2*dt * (1 - np.exp(-k*p))

def exponential_birth(p,dt):
    k = 0.001 - np.log(dt)/p_carrying
    return 1.5*dt * (1 - np.exp(-k*p))

def deer_recurrence(dn_1, pn_1, dt, b=linear_birth):
    # print(f"{dn_1} - {1.1*dn_1*dt} + {(1.5/p_carrying)*pn_1*dn_1 * dt}")
    return dn_1 - 1.1*dn_1*dt + b(pn_1,dt)*dn_1

def plant_recurrence(pn_1, dn_1, dt, E=linear_eating):
    # p(n) = p(n-1) + p(n-1) * g(p(n-1)) - d(n-1) * E(p(n-1))
    return pn_1 + pn_1 * (plant_r * (1 - pn_1 / p_carrying)) * dt - E(pn_1,dt) * dn_1

### No deer

In [ ]:
n_steps = 30*365
dt_1 = 1/365

# ^ this means we run for thirty years at intervals of one year

n = np.arange(n_steps)
d = np.empty_like(n)
p = np.empty_like(n)

d.dtype = float
p.dtype = float

p[0] = 1
d[0] = 0

for i in range(1,len(n)):
    p[i] = plant_recurrence(p[i-1],d[i-1], dt_1)
    d[i] = deer_recurrence(d[i-1],p[i-1], dt_1)

plt.plot(n*dt_1,p,label="plants",c="tab:green") 
# plt.plot(n*dt_1,d,label="deer",c="tab:brown")
plt.ylabel("Plant Density $p(n)$")
plt.xlabel("Year")
plt.title("Plant Population (no deer)")
# _=plt.legend() 

### Deer, linear, dt = 1

In [ ]:
n_steps = 30
dt_1 = 1

# ^ this means we run for thirty years at intervals of one year

n = np.arange(n_steps)
d = np.empty_like(n)
p = np.empty_like(n)

d.dtype = float
p.dtype = float

p[0] = p0
d[0] = d0

for i in range(1,len(n)):
    p[i] = plant_recurrence(p[i-1],d[i-1], dt_1)
    d[i] = deer_recurrence(d[i-1],p[i-1], dt_1)

fig, ax1 = plt.subplots()

plt.xlabel("Year")
ax1.set_ylabel('Deer', color=brown)
ax1.plot(n*dt_1, d, color=brown, label='deer')
ax1.tick_params(axis='y', labelcolor=brown)

ax2 = ax1.twinx()
ax2.set_ylabel('Plants', color=green)
ax2.plot(n*dt_1, p, color=green, label='plants')
ax2.tick_params(axis='y', labelcolor=green)

plt.title("Deer and Plant Population (dt = 1 year)")
plt.show()


### Deer, linear, dt = 1/365

In [ ]:
n_steps = 30*365
dt_2 = 1/365

# ^ this means we run for thirty years at intervals of one day

n_2 = np.arange(n_steps)
d_2 = np.empty_like(n_2)
p_2 = np.empty_like(n_2)

d_2.dtype = float
p_2.dtype = float

p_2[0] = p0
d_2[0] = d0

for i in range(1,len(n_2)):
    p_2[i] = plant_recurrence(p_2[i-1],d_2[i-1], dt_2)
    d_2[i] = deer_recurrence(d_2[i-1],p_2[i-1], dt_2)

fig, ax1 = plt.subplots()

plt.xlabel("Year")
ax1.set_ylabel('Deer', color=brown)
ax1.plot(n_2*dt_2, d_2, color=brown, label='deer')
ax1.tick_params(axis='y', labelcolor=brown)

ax2 = ax1.twinx()
ax2.set_ylabel('Plants', color=green)
ax2.plot(n_2*dt_2, p_2, color=green, label='plants')
ax2.tick_params(axis='y', labelcolor=green)

plt.title("Deer and Plant Population (dt = 1 day)")
plt.show()

In [9]:
def model(n_years, dt, birth_function, eat_function):
    n_steps = n_years/dt
    # ^ this means we run for n years at intervals of dt

    n = np.arange(n_steps)
    d = np.empty_like(n)
    p = np.empty_like(n)

    d.dtype = float
    p.dtype = float

    p[0] = p0
    d[0] = d0

    for i in range(1,len(n)):
        p[i] = plant_recurrence(p[i-1],d[i-1], dt, eat_function)
        d[i] = deer_recurrence(d[i-1],p[i-1], dt, birth_function)
    
    return n,p,d

In [ ]:
dt1 = 1
dt2 = 1/365

n1, p1, d1 = model(30, dt1, linear_birth, linear_eating)
n2, p2, d2 = model(30, dt2, linear_birth, linear_eating)

fig, ax1 = plt.subplots()

plt.xlabel("Year")
ax1.set_ylabel('Deer', color=brown)
l1 = ax1.plot(n1*dt1,d1,label="deer (dt = 1 year)",c="tab:brown",linestyle="dashed",alpha=0.75)
l2 = ax1.plot(n2*dt2,d2,label="deer (dt = 1 day)",c="tab:brown",alpha=0.75)
ax1.tick_params(axis='y', labelcolor=brown)

ax2 = ax1.twinx()
ax2.set_ylabel('Plants', color=green)
l3 = ax2.plot(n1*dt1,p1,label="plants (dt = 1 year)",c="tab:green",linestyle="dashed",alpha=0.75) 
l4 = ax2.plot(n2*dt2,p2,label="plants (dt = 1 day)",c="tab:green",alpha=0.75) 
ax2.tick_params(axis='y', labelcolor=green)

leg1 = ax1.legend([l1[0], l2[0]], ["deer (dt = 1 year)", "deer (dt = 1 day)"], loc='center left', bbox_to_anchor=(0.5, 0.5), frameon=False)
leg2 = ax2.legend([l3[0], l4[0]], ["plants (dt = 1 year)", "plants (dt = 1 day)"], loc='center left', bbox_to_anchor=(0.5, 0.4), frameon=False)

plt.title("Deer and Plant Population")
plt.show()


#######

savefig("population_1y_1d_linear.png")

In [ ]:
dt1 = 1
dt2 = 1/365

n1, p1, d1 = model(100, dt1, linear_birth, linear_eating)
n2, p2, d2 = model(100, dt2, linear_birth, linear_eating)

fig, ax1 = plt.subplots()

plt.xlabel("Year")
ax1.set_ylabel('Deer', color=brown)
l1 = ax1.plot(n1*dt1,d1,label="deer (dt = 1 year)",c="tab:brown",linestyle="dashed",alpha=0.75)
l2 = ax1.plot(n2*dt2,d2,label="deer (dt = 1 day)",c="tab:brown",alpha=0.75)
ax1.tick_params(axis='y', labelcolor=brown)

ax2 = ax1.twinx()
ax2.set_ylabel('Plants', color=green)
l3 = ax2.plot(n1*dt1,p1,label="plants (dt = 1 year)",c="tab:green",linestyle="dashed",alpha=0.75) 
l4 = ax2.plot(n2*dt2,p2,label="plants (dt = 1 day)",c="tab:green",alpha=0.75) 
ax2.tick_params(axis='y', labelcolor=green)

leg1 = ax1.legend([l1[0], l2[0]], ["deer (dt = 1 year)", "deer (dt = 1 day)"], loc='center left', bbox_to_anchor=(0.5, 0.5), frameon=False)
leg2 = ax2.legend([l3[0], l4[0]], ["plants (dt = 1 year)", "plants (dt = 1 day)"], loc='center left', bbox_to_anchor=(0.5, 0.4), frameon=False)

plt.title("Deer and Plant Population")
plt.show()


#######

# savefig("population_1y_1d_linear.png")

In [ ]:
print(f"Deer: mean={np.mean(d2[-50:])}, med={np.median(d2[-50:])}")
print(f"Plants: mean={np.mean(p2[-50:])}, med={np.median(p2[-50:])}")

In [ ]:
print(f"Deer: mean={np.mean(d1[-50:])}, med={np.median(d1[-50:])}")
print(f"Plants: mean={np.mean(p1[-50:])}, med={np.median(p1[-50:])}")

In [ ]:
plt.plot(d,p)
plt.ylabel("Plant Density")
plt.xlabel("Deer Population")
plt.title("Phase Plane: dt = 1 year")
savefig("phase_plane_1_year_linear.png")

In [ ]:
plt.plot(d_2,p_2)
plt.ylabel("Plant Density")
plt.xlabel("Deer Population")
plt.title("Phase Plane: dt = 1 day")
savefig("phase_plane_1_day_linear.png")

In [ ]:
plt.plot(d,p, label="dt = 1 year",color="tab:blue",linestyle="dashed")
plt.plot(d_2,p_2,label="dt = 1 day",color="tab:blue")

plt.ylabel("Plant Density")
plt.xlabel("Deer Population")
plt.title("Phase Plane")
plt.legend()
savefig("phase_plane_both_linear.png")

# Part 2

In [ ]:
dt = 1/365
duration = 100 # years

n1, p1, d1 = model(duration, dt, linear_birth, linear_eating)
n2, p2, d2 = model(duration, dt, exponential_birth, exponential_eating)

fig, ax1 = plt.subplots()

plt.xlabel("Year")
ax1.set_ylabel('Deer', color=brown)
l1 = ax1.plot(n1*dt,d1,label="deer (linear)",c="tab:brown",linestyle="dashed",alpha=0.75)
l2 = ax1.plot(n2*dt,d2,label="deer (exponential)",c="tab:brown",alpha=0.75)
ax1.tick_params(axis='y', labelcolor=brown)

ax2 = ax1.twinx()
ax2.set_ylabel('Plants', color=green)
l3 = ax2.plot(n1*dt,p1,label="plants (linear)",c="tab:green",linestyle="dashed",alpha=0.75) 
l4 = ax2.plot(n2*dt,p2,label="plants (exponential)",c="tab:green",alpha=0.75) 
ax2.tick_params(axis='y', labelcolor=green)

leg1 = ax1.legend([l1[0], l2[0], l3[0],l4[0]], ["d (lin)", "d (exp)", "p (lin)", "p (exp)"], loc='upper center',  ncol=4)

# leg1 = ax1.legend([l1[0], l2[0]], ["deer (linear)", "deer (exponential)"], loc='center left', bbox_to_anchor=(0.5, 0.5), frameon=False)
# leg2 = ax2.legend([l3[0], l4[0]], ["plants (linear)", "plants (exponential)"], loc='center left', bbox_to_anchor=(0.5, 0.4), frameon=False)

plt.title("Deer and Plant Population (dt = 1 day)")

xlims_1day_exp = ax1.get_xlim()
ylims_1day_exp = ax1.get_ylim()
ylims_2_1day_exp = ax2.get_ylim()
######

# plt.plot(n1*dt,p1,label="plants (linear)",c="tab:green",linestyle="dashed",alpha=0.75) 
# plt.plot(n1*dt,d1,label="deer (linear)",c="tab:brown",linestyle="dashed",alpha=0.75)

# plt.plot(n2*dt,p2,label="plants (exponential)",c="tab:green",alpha=0.75) 
# plt.plot(n2*dt,d2,label="deer (exponential)",c="tab:brown",alpha=0.75)

# plt.ylabel("Plant Density | Deer Population")
# plt.xlabel("Year")
# plt.title("Deer and Plant Population (dt = 1 day)")
# _=plt.legend() 
savefig("exponential_vs_linear_1_day.png")

In [ ]:
plt.plot(d2,p2)
plt.ylabel("Plant Density")
plt.xlabel("Deer Population")
plt.title("Phase Plane: dt = 1 day (exponential)")
savefig("phase_plane_1_day_exp.png")

In [17]:
import matplotlib.animation as animation

In [ ]:
fig,ax1 = plt.subplots()
ax1.set_xlim(*xlims_1day_exp)
ax1.set_ylim(*ylims_1day_exp)
ax2 = ax1.twinx()

def anim(frame):
    frame = int(frame/dt)
    ax1.cla()

    ax1.set_xlabel("Year")
    ax1.set_ylabel('Deer', color=brown)
    l1 = ax1.plot(n1[:frame]*dt,d1[:frame],label="deer (linear)",c="tab:brown",linestyle="dashed",alpha=0.75)
    l2 = ax1.plot(n2[:frame]*dt,d2[:frame],label="deer (exponential)",c="tab:brown",alpha=0.75)
    ax1.tick_params(axis='y', labelcolor=brown)

    ax1.set_xlim(*xlims_1day_exp)
    ax1.set_ylim(*ylims_1day_exp)
    ax2.set_ylim(*ylims_2_1day_exp)

    # ax2 = ax1.twinx()
    ax2.set_ylabel('Plants', color=green)
    l3 = ax2.plot(n1[:frame]*dt,p1[:frame],label="plants (linear)",c="tab:green",linestyle="dashed",alpha=0.75) 
    l4 = ax2.plot(n2[:frame]*dt,p2[:frame],label="plants (exponential)",c="tab:green",alpha=0.75) 
    ax2.tick_params(axis='y', labelcolor=green)
    ax1.set_title("Deer and Plant Population (dt = 1 day)")
    leg1 = ax1.legend([l1[0], l2[0], l3[0],l4[0]], ["d (lin)", "d (exp)", "p (lin)", "p (exp)"], loc='upper center',  ncol=4)
    return [l1,l2,l3,l4,leg1]

ani = animation.FuncAnimation(fig=fig, func=anim, frames=100, interval=75, repeat=False)
ani.save(filename=out("deer_1day_exp_lin.gif"), writer="pillow",dpi=150)


## dt = 1 year

In [ ]:
dt = 1
duration = 100 # years

n1, p1, d1 = model(duration, dt, linear_birth, linear_eating)
n2, p2, d2 = model(duration, dt, exponential_birth, exponential_eating)

fig, ax1 = plt.subplots()

plt.xlabel("Year")
ax1.set_ylabel('Deer', color=brown)
l1 = ax1.plot(n1*dt,d1,label="deer (linear)",c="tab:brown",linestyle="dashed",alpha=0.75)
l2 = ax1.plot(n2*dt,d2,label="deer (exponential)",c="tab:brown",alpha=0.75)
ax1.tick_params(axis='y', labelcolor=brown)

ax2 = ax1.twinx()
ax2.set_ylabel('Plants', color=green)
l3 = ax2.plot(n1*dt,p1,label="plants (linear)",c="tab:green",linestyle="dashed",alpha=0.75) 
l4 = ax2.plot(n2*dt,p2,label="plants (exponential)",c="tab:green",alpha=0.75) 
ax2.tick_params(axis='y', labelcolor=green)

# leg1 = ax1.legend([l1[0], l2[0]], ["deer (linear)", "deer (exponential)"], loc='center left', bbox_to_anchor=(0.5, 0.5), frameon=False)
leg1 = ax1.legend([l1[0], l2[0], l3[0],l4[0]], ["d (lin)", "d (exp)", "p (lin)", "p (exp)"], loc='upper center',  ncol=4)
# leg2 = ax2.legend([l3[0], l4[0]], ["plants (linear)", "plants (exponential)"], loc='center left', bbox_to_anchor=(0.5, 0.4), frameon=False)

xlims_1year_exp = ax1.get_xlim()
ylims_1year_exp = ax1.get_ylim()
ylims_2_1year_exp = ax2.get_ylim()

plt.title("Deer and Plant Population (dt = 1 year)")

###

# plt.plot(n1*dt,p1,label="plants (linear)",c="tab:green",linestyle="dashed",alpha=0.75) 
# plt.plot(n1*dt,d1,label="deer (linear)",c="tab:brown",linestyle="dashed",alpha=0.75)

# plt.plot(n2*dt,p2,label="plants (exponential)",c="tab:green",alpha=0.75) 
# plt.plot(n2*dt,d2,label="deer (exponential)",c="tab:brown",alpha=0.75)

# plt.ylabel("Plant Density | Deer Population")
# plt.xlabel("Year")
# plt.title("Deer and Plant Population (dt = 1 year)")
# _=plt.legend()
savefig("exponential_vs_linear_1_year.png")

In [ ]:
plt.plot(d2,p2)
plt.ylabel("Plant Density")
plt.xlabel("Deer Population")
plt.title("Phase Plane: dt = 1 year (exponential)")
savefig("phase_plane_1_year_exp.png")

In [ ]:
print(d2)
print(p2)

In [ ]:
fig,ax1 = plt.subplots()
ax1.set_xlim(*xlims_1year_exp)
ax1.set_ylim(*ylims_1year_exp)
ax2 = ax1.twinx()

def anim(frame):
    ax1.cla()

    ax1.set_xlabel("Year")
    ax1.set_ylabel('Deer', color=brown)
    l1 = ax1.plot(n1[:frame]*dt,d1[:frame],label="deer (linear)",c="tab:brown",linestyle="dashed",alpha=0.75)
    l2 = ax1.plot(n2[:frame]*dt,d2[:frame],label="deer (exponential)",c="tab:brown",alpha=0.75)
    ax1.tick_params(axis='y', labelcolor=brown)

    ax1.set_xlim(*xlims_1year_exp)
    ax1.set_ylim(*ylims_1year_exp)
    ax2.set_ylim(*ylims_2_1year_exp)

    # ax2 = ax1.twinx()
    ax2.set_ylabel('Plants', color=green)
    l3 = ax2.plot(n1[:frame]*dt,p1[:frame],label="plants (linear)",c="tab:green",linestyle="dashed",alpha=0.75) 
    l4 = ax2.plot(n2[:frame]*dt,p2[:frame],label="plants (exponential)",c="tab:green",alpha=0.75) 
    ax2.tick_params(axis='y', labelcolor=green)
    ax1.set_title("Deer and Plant Population (dt = 1 year)")
    leg1 = ax1.legend([l1[0], l2[0], l3[0],l4[0]], ["d (lin)", "d (exp)", "p (lin)", "p (exp)"], loc='upper center',  ncol=4)
    return [l1,l2,l3,l4,leg1]

ani = animation.FuncAnimation(fig=fig, func=anim, frames=100, interval=75, repeat=False)
ani.save(filename=f"deer_1year_exp_lin.gif", writer="pillow",dpi=150)


In [ ]:
r = 1.5

def exp(x,r): return r*(1-np.exp(-0.001*x)) 
def lin(x,r): return (r / 3000) * x
x=np.linspace(0,5000,10000)
plt.plot(x,exp(x,r))
plt.plot(x,lin(x,r))
ylims = plt.ylim()
xlims = plt.xlim()
plt.vlines(3000,*ylims,linestyles="dashed",color="tab:green")
plt.hlines(r,*xlims,linestyles="dashed",color="tab:green")
plt.ylim(ylims)
plt.xlim(xlims)
plt.title(f"Exponential vs linear rate, r={r}")